# Playground Series 5.06 🦆

# Imports

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

## Read Dataset

In [18]:
train = pd.read_csv("data/train.csv", index_col=  "id")
test = pd.read_csv("data/test.csv", index_col=  "id")
print(train.shape)
train.head()

(750000, 9)


,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
id,,,,,,,,,
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,27,69,65,Sandy,Millets,30,6,18,28-28
2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,35,58,43,Red,Paddy,37,2,16,DAP


## Feature Engineering

In [19]:
def feature_engineering(df):

    
    label_encoders = {}
    for col in ['Soil Type', 'Crop Type', 'Fertilizer Name']:
        if (col == "Fertilizer Name") & ("Fertilizer Name" not in df.columns):
            continue
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    return df

train = feature_engineering(train)
test = feature_engineering(test)

In [20]:
# Split features and target
X_train = train.drop('Fertilizer Name', axis=1)
y_train = train['Fertilizer Name']

X_test = test

# Scale features
scaler = StandardScaler()
X__train_scaled = scaler.fit_transform(X_train)
X__test_scaled = scaler.fit_transform(X_test)

# One-hot encode target
y_train_cat = to_categorical(y_train)

## Build Model

In [21]:
model = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu'),
    Dense(16, activation='relu'),
    Dense(y_train_cat.shape[1], activation='softmax')
])

/home/rique/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1750427082.712709    6055 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9502 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:05:00.0, compute capability: 8.9
2025-06-20 14:44:43.312580: W external/local_xla/xla/service/gpu/llvm_gpu_backend/default/nvptx_libdevice_path.cc:40] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/

In [22]:
# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=50, batch_size=4, validation_data=(X_test, y_test))

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

NameError: name 'y_test' is not defined